In [1]:
%matplotlib inline


# Clustering text documents using k-means


This is an example showing how the scikit-learn can be used to cluster
documents by topics using a bag-of-words approach. This example uses
a scipy.sparse matrix to store the features instead of standard numpy arrays.

Two feature extraction methods can be used in this example:

  - TfidfVectorizer uses a in-memory vocabulary (a python dict) to map the most
    frequent words to features indices and hence compute a word occurrence
    frequency (sparse) matrix. The word frequencies are then reweighted using
    the Inverse Document Frequency (IDF) vector collected feature-wise over
    the corpus.

  - HashingVectorizer hashes word occurrences to a fixed dimensional space,
    possibly with collisions. The word count vectors are then normalized to
    each have l2-norm equal to one (projected to the euclidean unit-ball) which
    seems to be important for k-means to work in high dimensional space.

    HashingVectorizer does not provide IDF weighting as this is a stateless
    model (the fit method does nothing). When IDF weighting is needed it can
    be added by pipelining its output to a TfidfTransformer instance.

Two algorithms are demoed: ordinary k-means and its more scalable cousin
minibatch k-means.

Additionally, latent semantic analysis can also be used to reduce dimensionality
and discover latent patterns in the data. 

It can be noted that k-means (and minibatch k-means) are very sensitive to
feature scaling and that in this case the IDF weighting helps improve the
quality of the clustering by quite a lot as measured against the "ground truth"
provided by the class label assignments of the 20 newsgroups dataset.

This improvement is not visible in the Silhouette Coefficient which is small
for both as this measure seem to suffer from the phenomenon called
"Concentration of Measure" or "Curse of Dimensionality" for high dimensional
datasets such as text data. Other measures such as V-measure and Adjusted Rand
Index are information theoretic based evaluation scores: as they are only based
on cluster assignments rather than distances, hence not affected by the curse
of dimensionality.

Note: as k-means is optimizing a non-convex objective function, it will likely
end up in a local optimum. Several runs with independent random init might be
necessary to get a good convergence.



In [16]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import numpy as np
import pandas as pd

Load some categories from the training set


In [24]:
df = pd.read_pickle('../data/feature_dumps/features.pkl')
blogsM = df[df.Gender == 'M'].Blog.values.astype(str)
blogsF = df[df.Gender == 'F'].Blog.values.astype(str)

n_features = 20000

print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()

vectorizerM = TfidfVectorizer(max_df=0.5, max_features=n_features,
                         min_df=2, stop_words='english',
                         use_idf=True)
vectorizerF = TfidfVectorizer(max_df=0.5, max_features=n_features,
                         min_df=2, stop_words='english',
                         use_idf=True)

XM = vectorizerM.fit_transform(blogsM)
XF = vectorizerF.fit_transform(blogsF)

print("done in %fs" % (time() - t0))
print("\nn_samples: %d, n_features: %d" % XM.shape)
print("\nn_samples: %d, n_features: %d" % XF.shape)
print()

Extracting features from the training dataset using a sparse vectorizer
done in 1.452787s

n_samples: 1671, n_features: 17308

n_samples: 1541, n_features: 15361



Do the actual clustering


In [25]:
n_clusters = 20

kmminiM = MiniBatchKMeans(n_clusters=n_clusters, init='k-means++', n_init=1,
                     init_size=1000, batch_size=1000, verbose=False)
kmminiF = MiniBatchKMeans(n_clusters=n_clusters, init='k-means++', n_init=1,
                     init_size=1000, batch_size=1000, verbose=False)

kmM = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1, verbose=False)
kmF = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1, verbose=False)

t0 = time()
kmM.fit(XM)
kmF.fit(XF)

kmminiM.fit(XM)
kmminiF.fit(XF)

print("done in %0.3fs" % (time() - t0))
print()
print("Top terms per cluster:")

order_centroids_M = kmM.cluster_centers_.argsort()[:, ::-1]
order_centroids_mini_M = kmminiM.cluster_centers_.argsort()[:, ::-1]
order_centroids_F = kmF.cluster_centers_.argsort()[:, ::-1]
order_centroids_mini_F = kmminiF.cluster_centers_.argsort()[:, ::-1]

termsM = vectorizerM.get_feature_names()
termsF = vectorizerF.get_feature_names()

print("\nMales:")
print("KMeans:")
for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_M[i, :10]:
        print(' %s' % termsM[ind], end='')
    print()
print()
print("MiniBatch:")
for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_mini_M[i, :10]:
        print(' %s' % termsM[ind], end='')
    print()

print("\nFemales:")
print("KMeans:")
for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_F[i, :10]:
        print(' %s' % termsF[ind], end='')
    print()
print()
print("MiniBatch:")
for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_mini_F[i, :10]:
        print(' %s' % termsF[ind], end='')
    print()

done in 7.232s

Top terms per cluster:

Males:
KMeans:
Cluster 0: em youtube band saxophone nodes just prototyping video tools mp3
Cluster 1: like food wid days ready wud im person best mr
Cluster 2: just ice time week home day work hard sick told
Cluster 3: data don paper like kosovo think conference use serbia model
Cluster 4: god faith does sin things christ hide heaven bless jesus
Cluster 5: love heart know just like world don man think want
Cluster 6: person gud college tht talk nakkals guy pain life fan
Cluster 7: friend guy best good lol ma gud person class frnd
Cluster 8: blog post new com site cancer video ve twitter ll
Cluster 9: like new just time year day good did got people
Cluster 10: nbsp just lake people blog 2009 like python world skywatch
Cluster 11: game games play metroid team wii mario time tournament just
Cluster 12: relationship partner feel problems time really need life csi end
Cluster 13: women men fat muscle skin excess burn higher woman mass
Cluster 14: goog